# Genscape API: Download all data to cvs and create xlsx pivot file

## Staging

In [1]:
import pandas as pd
import os
import datetime
import numpy as np

pd.options.mode.chained_assignment = None
APIKEY = 'INPUT HERE'


In [2]:
def GenscapeApiHistory(url, ApiKey):

    #Example ApiData = GenscapeApiHistory("https://api.genscape.com/transportation/oil/v1/pipeline-flows/weekly?startDate=2019-01-01","abcde123456aa84848")

    import json
    import http.client
    import urllib.request
    import urllib.parse
    import urllib.error
    import time

    urlPieces = url.partition('?')
    urlParameters = urlPieces[2]

    if 'limit' in urlParameters or 'offset' in urlParameters or 'format' in urlParameters:

        print("Please remove any offset, limit or format parameters from your API URL and try again")

    else:

        limit = 5000
        offset = 0
        i = 1
        data = []
        while True:

            headers = {

                'Gen-Api-Key': str(ApiKey),
            }

            params = urllib.parse.urlencode({
                'limit': limit,
                'offset': offset,
                'format': 'json',
            })

            conn = http.client.HTTPSConnection('api.genscape.com')

            conn.request("GET", str(url)+"&%s" % params, "{body}", headers)
            response = conn.getresponse()
            decoded_data = response.read()
            decoded_data2 = json.loads(decoded_data)
            try:
                decoded_data3 = decoded_data2['data']
            except:
                print(
                    "Empty Dataset: Please confirm your API URL and API Key are correct")
            data = data + decoded_data3
            recvd = len(decoded_data3)
            recvdTotal = len(data)
            print("API call # " + str(i) + " returned " + str(recvd) +
                  " rows for a total of " + str(recvdTotal))
            if recvd < limit:
                break
            offset += limit
            i = i+1
            time.sleep(2.1)
        print("API returned a total of " + str(len(data)) + " rows")

        conn.close()

        return(data)

In [3]:
def GenscapeApi(url, ApiKey):

    #Example ApiData = GenscapeApi("https://api.genscape.com/transportation/oil/v1/pipeline-flows/weekly?startDate=2019-01-01","abcde123456aa84848")

    import json
    import http.client
    import urllib.request
    import urllib.parse
    import urllib.error

    urlPieces = url.partition('?')
    urlParameters = urlPieces[2]

    if 'format' in urlParameters:

        print("Please remove the format parameters from your API URL and try again")

    else:

        data = []
        while True:

            headers = {
                'Gen-Api-Key': str(ApiKey),
            }

            conn = http.client.HTTPSConnection('api.genscape.com')
            conn.request("GET", str(url), "{body}", headers)
            response = conn.getresponse()
            decoded_data = response.read()
            decoded_data2 = json.loads(decoded_data)
            try:
                decoded_data3 = decoded_data2['data']
            except:
                print(
                    "Empty Dataset: Please confirm your API URL and API Key are correct")
            data = data + decoded_data3
            recvd = len(decoded_data3)
            print("API returned " + str(recvd) + " rows")
            break

        conn.close()

        return(data)

In [4]:
def url_list():
    df_url = pd.DataFrame({'name': ['df_pipe_mo_flow',
                                    'df_pipe_day_flow',
                                    'df_pipe_half_hourly',
                                    'df_pipe_wk_netflow',
                                    'df_pipe_day_netflow',
                                    'df_rail_mo_flow',
                                    'df_rail_day_flow',
                                    'df_regs',
                                    'df_meta',
                                    'df_stor_reg',
                                    'df_stor_own',
                                    'df_stor_tank'
                                    ],
                           'url': ['https://api.genscape.com/transportation/oil/v2/pipeline-flows/monthly?revision=revised&startDate=2017-01-01',
                                   'https://api.genscape.com/transportation/oil/v2/pipeline-flows/daily?revision=revised&startDate=2017-01-01',
                                   'https://api.genscape.com/transportation/oil/v2/pipeline-flows/half-hourly?revision=revised&startDate=2017-01-01',
                                   'https://api.genscape.com/transportation/oil/v1/pipeline-flows/net/cushing/weekly?revision=revised&startDate=2021-01-01',
                                   'https://api.genscape.com/transportation/oil/v1/pipeline-flows/net/cushing/daily?revision=revised&startDate=2021-01-01',
                                   'https://api.genscape.com/transportation/oil/v2/rail-flows/monthly?startDate=2017-01-01',
                                   'https://api.genscape.com/transportation/oil/v2/rail-flows/daily?startDate=2017-01-01',
                                   'https://api.genscape.com/transportation/oil/v1/pipeline-flows/regulatory?startDate=2017-01-01',
                                   'https://api.genscape.com/transportation/oil/v1/pipelines',
                                   'https://api.genscape.com/storage/oil/v1/region-volumes?revision=published&startDate=2017-01-01',
                                   'https://api.genscape.com/storage/oil/v1/owner-volumes?revision=published&startDate=2017-01-01',
                                   'https://api.genscape.com/storage/oil/v1/tank-volumes?revision=published&startDate=2017-01-01',

                                   ]})
    return df_url


df_url = url_list()


In [19]:
df_url.to_csv('asdf.csv')

## Download Data

#### Pipeline Half-hour data

In [5]:
def pipe_halfhourly():
    days_before = (datetime.date.today() -
                   datetime.timedelta(days=2)).isoformat()
    url = 'https://api.genscape.com/transportation/oil/v2/pipeline-flows/half-hourly?revision=revised&startDate=' + days_before
    df = pd.DataFrame(GenscapeApiHistory(url, APIKEY))

    date_format = "%Y-%m-%dT%H:%M:%SZ"
    df['reportDate'] = df['reportDate'].apply(
        lambda x: datetime.datetime.strptime(x, date_format))

    df['reportDate'] = df['reportDate'] + \
        pd.tseries.offsets.DateOffset(hours=-5)

    df['year'] = df['reportDate'].dt.year
    df['month'] = df['reportDate'].dt.month
    df['day'] = df['reportDate'].dt.day
    df['hour'] = 0
    df['minute'] = 0
    df['second'] = 0

    df['date'] = pd.to_datetime(
        df[['year', 'month', 'day', 'hour', 'minute', 'second']]).dt.date.astype('datetime64')

    col = df.pop('date')
    df.insert(0, col.name, col)
    df = df.iloc[:, :-6]

    df = df[df['date'] > min(df['date'])]
    df1 = df
    df1.to_csv('pipe_halfhourly.csv', index=False)
    return df1


df1 = pipe_halfhourly()


API call # 1 returned 5000 rows for a total of 5000
API call # 2 returned 5000 rows for a total of 10000
API call # 3 returned 3626 rows for a total of 13626
API returned a total of 13626 rows


#### Pipeline Daily data

In [6]:
def pipe_daily():
    days_before = (datetime.date.today() -
                   datetime.timedelta(days=45)).isoformat()
    url = 'https://api.genscape.com/transportation/oil/v2/pipeline-flows/daily?revision=revised&startDate=' + days_before
    df = pd.DataFrame(GenscapeApiHistory(url, APIKEY))

    df['reportDate'] = pd.to_datetime(
        df['reportDate']).dt.date.astype('datetime64')
    df2 = df
    df2.to_csv('pipe_daily.csv', index=False)
    return days_before, df2


days_before, df2 = pipe_daily()


API call # 1 returned 4590 rows for a total of 4590
API returned a total of 4590 rows


#### Pipeline Monthly data

In [7]:
def pipe_monthly():
    url = 'https://api.genscape.com/transportation/oil/v2/pipeline-flows/monthly?revision=revised&startDate=2017-01-01'
    df = pd.DataFrame(GenscapeApiHistory(url, APIKEY))

    df['reportDate'] = pd.to_datetime(
        df['reportDate']).dt.date.astype('datetime64')
    df3 = df
    df3.to_csv('pipe_monthly.csv', index=False)
    return df3


df3 = pipe_monthly()


API call # 1 returned 5000 rows for a total of 5000
API call # 2 returned 1733 rows for a total of 6733
API returned a total of 6733 rows


#### Combine half-hourly+daily data to create MTD pipe flow

In [8]:
def pipe_hourly_todaily():
    #df1 = pipe_halfhourly() #to get 3 days worth of data
    #days_before, df2 = pipe_daily() #to get x 60 to 180 days worth of data
    #df3 = pipe_monthly() #to get data since 1/1/17
    df4 = pd.DataFrame()  # to add the halfhourly data to the daily data and create a new table

    #get data to add to daily table
    df = df1[df1['date'] > max(df2['reportDate'])]
    df.drop('reportDate', axis=1, inplace=True)
    df.rename(columns={'date': 'reportDate'}, inplace=True)
    col_list_value = ['pipelineCapacity', 'flowBpd', 'confidenceId']
    df = df.groupby(['reportDate', 'pipelineId', 'pipelineName', 'regionName', 'startPumpStation',
                    'finishPumpStation', 'diameter', 'direction'])[col_list_value].mean()
    df = df.reset_index()

    df4 = df2.append(df, ignore_index=True)

    df4.to_csv('pipe_daily&halfhourly.csv', index=False)
    return df4


df4 = pipe_hourly_todaily()


def pipe_daily_add_to_master():
    filename_path = 'df_pipe_daily_master.csv'
    if not os.path.exists(filename_path):
        print('### NEED TO CREATE DAILY MASTER FILE ###')
        url = 'https://api.genscape.com/transportation/oil/v2/pipeline-flows/daily?revision=revised&startDate=2017-01-01'
        df = pd.DataFrame(GenscapeApiHistory(url, APIKEY))
        df.to_csv(filename_path, index=False)
    else:
        df = pd.read_csv(filename_path)
    df['reportDate'] = pd.to_datetime(
        df['reportDate'], format="%Y-%m-%d").astype('datetime64')

    days_before_dt = datetime.datetime.fromisoformat(days_before)

    df = df[df['reportDate'] < days_before_dt]

    df = df.append(df4, ignore_index=True)
    df5 = df
    df5.to_csv(filename_path, index=False)
    return df5


df5 = pipe_daily_add_to_master()


def pipe_monthly_add_to_master():
    df = df5
    df['year'] = df['reportDate'].dt.year
    df['month'] = df['reportDate'].dt.month
    df['day'] = 1
    df['hour'] = 0
    df['minute'] = 0
    df['second'] = 0

    df['date'] = pd.to_datetime(
        df[['year', 'month', 'day', 'hour', 'minute', 'second']]).dt.date.astype('datetime64')

    col = df.pop('date')
    df.insert(0, col.name, col)
    df = df.iloc[:, :-6]
    df.drop('reportDate', axis=1, inplace=True)
    df.rename(columns={'date': 'reportDate'}, inplace=True)
    col_list_value = ['pipelineCapacity', 'flowBpd', 'confidenceId']
    df = df.groupby(['reportDate', 'pipelineId', 'pipelineName', 'regionName', 'startPumpStation',
                    'finishPumpStation', 'diameter', 'direction'])[col_list_value].mean()
    df = df.reset_index()
    df = df[df['reportDate'] > max(df3['reportDate'])]
    df6 = df3.append(df, ignore_index=True)
    df6.to_csv('df_pipe_monthly_master.csv', index=False)
    return df6


df6 = pipe_monthly_add_to_master()


### Cushing Netflow

In [9]:
def df_cushing_netflow():
    url = 'https://api.genscape.com/transportation/oil/v1/pipeline-flows/net/cushing/weekly?revision=revised&startDate=2017-01-01'
    df_weekly = pd.DataFrame(GenscapeApiHistory(url, APIKEY))
    df_weekly.to_csv('df_cush_netflow_weekly.csv', index=False)
    url = 'https://api.genscape.com/transportation/oil/v1/pipeline-flows/net/cushing/daily?revision=revised&startDate=2017-01-01'
    df_daily = pd.DataFrame(GenscapeApiHistory(url, APIKEY))
    df_daily.to_csv('df_cush_netflow_daily.csv', index=False)

    return df_weekly, df_daily


df_cushnet_weekly, df_cushnet_daily = df_cushing_netflow()


API call # 1 returned 318 rows for a total of 318
API returned a total of 318 rows
API call # 1 returned 2229 rows for a total of 2229
API returned a total of 2229 rows


### Regulatory Data

In [10]:
def df_regulatory():
    url = 'https://api.genscape.com/transportation/oil/v1/pipeline-flows/regulatory?startDate=2017-01-01'
    df_reg = pd.DataFrame(GenscapeApiHistory(url, APIKEY))
    df_reg['reportStartDate'] = pd.to_datetime(df_reg['reportStartDate'])

    df_reg_qrt = df_reg
    df_reg_qrt = df_reg_qrt[df_reg_qrt['frequency'] == 'Quarterly']
    for i in range(2):
        df_reg_qrt['reportStartDate'] = (df_reg_qrt['reportStartDate'].dt.floor(
            'd') + pd.offsets.MonthEnd(2) - pd.offsets.MonthBegin(1))
        df_reg = df_reg.append(df_reg_qrt)
    df_reg.to_csv('df_regulatory.csv', index=False)
    return df_reg


df_reg = df_regulatory()


API call # 1 returned 4251 rows for a total of 4251
API returned a total of 4251 rows


### Storage Data

#### Storage by Region

In [11]:
def df_storage_region():
    url = 'https://api.genscape.com/storage/oil/v1/region-volumes?revision=published&startDate=2017-01-01'
    df_stor_reg = pd.DataFrame(GenscapeApiHistory(url, APIKEY))
    df_stor_reg['reportDate'] = pd.to_datetime(df_stor_reg['reportDate'])
    df_stor_reg.to_csv('df_Storage_Region.csv', index=False)
    return df_stor_reg


df_stor_reg = df_storage_region()


API call # 1 returned 3816 rows for a total of 3816
API returned a total of 3816 rows


#### Storage by Owner

In [12]:
def df_storage_owner():
    filename_path = 'df_storage_owner_master.csv'
    if not os.path.exists(filename_path):
        print('### NEED TO CREATE DAILY MASTER FILE ###')
        url = 'https://api.genscape.com/storage/oil/v1/owner-volumes?revision=published&startDate=2017-01-01'
        df = pd.DataFrame(GenscapeApiHistory(url, APIKEY))
        df.to_csv(filename_path, index=False)
    else:
        df = pd.read_csv(filename_path)
    df['reportDate'] = pd.to_datetime(
        df['reportDate'], format="%Y-%m-%d").astype('datetime64')
    days_before_dt = datetime.datetime.fromisoformat(days_before)
    df = df[df['reportDate'] < days_before_dt]

    df.dtypes
    df_stor_own = df

    url = 'https://api.genscape.com/storage/oil/v1/owner-volumes?revision=published&startDate=' + days_before
    df = pd.DataFrame(GenscapeApiHistory(url, APIKEY))
    df['reportDate'] = pd.to_datetime(
        df['reportDate'], format="%Y-%m-%d").astype('datetime64')
    df.to_csv('storage_owner.csv', index=False)
    df.dtypes
    #maxday_df = max(df['reportDate'])
    #minday_df = min(df['reportDate'])

    df_temp = df_stor_own.append(df, ignore_index=True)
    df = df_temp
    df['reportDate'] = pd.to_datetime(
        df['reportDate'], format="%Y-%m-%d").astype('datetime64')
    df.dtypes
    df.to_csv(filename_path, index=False)

    return df


df_stor_own = df_storage_owner()


API call # 1 returned 1128 rows for a total of 1128
API returned a total of 1128 rows


### Meta data

In [13]:
def df_meta():
    url = 'https://api.genscape.com/transportation/oil/v1/pipelines'
    df_meta = pd.DataFrame(GenscapeApi(url, APIKEY))
    df_meta['onlineDate'] = pd.to_datetime(
        df_meta['onlineDate'], errors='coerce').dt.date
    df_meta['effectiveDate'] = pd.to_datetime(
        df_meta['effectiveDate'], errors='coerce').dt.date
    df_meta.to_csv('df_PipelineMetadata.csv', index=False)
    return df_meta


df_meta = df_meta()


API returned 84 rows


## Combine all data into a pivot to create xlsx file

### xlsx file with all data

In [14]:
def direction_reclass(x, y, z):
    print(x, y, z)
    df['flowBpd'] = np.where((df['pipelineName'] == x) & (
        df['direction'] == y), df['flowBpd']*1, df['flowBpd']*-1)
    df['confidenceId'] = np.where((df['pipelineName'] == x) & (
        df['direction'] == y), df['confidenceId']*1, df['confidenceId']*-1)
    df['pipelineCapacity'] = np.where((df['pipelineName'] == x) & (
        df['direction'] == y), df['pipelineCapacity']*1, df['pipelineCapacity']*-1)

    df['direction'] = np.where((df['pipelineName'] == x) & (
        df['direction'] == y), z, df['direction'])
    return df


df = df6.copy()
df = direction_reclass('Red River', 'Cushing Outgoing', 'Cushing Incoming')
df = direction_reclass('Capline - Patoka to St. James',
                       'Other', 'Patoka Outgoing')
df10 = df.copy()


Red River Cushing Outgoing Cushing Incoming
Capline - Patoka to St. James Other Patoka Outgoing


In [15]:
GENSCAPE_XLSX = 'genscape_data.xlsx'
writer = pd.ExcelWriter(GENSCAPE_XLSX, engine='xlsxwriter')

pivot_reg = df_reg.pivot_table(index=['regionName', 'pipelineName', 'source', 'frequency',
                               'description'], columns=df_reg['reportStartDate'].dt.date, values='value').reset_index()
pivot_reg.to_excel(writer, sheet_name='Reg', index=False)

pivot_month_pipe_flow = df10.pivot_table(index=['regionName', 'pipelineName', 'direction', 'startPumpStation',
                                         'finishPumpStation'], columns=df10['reportDate'].dt.date, values='flowBpd').reset_index()
pivot_month_pipe_flow.to_excel(writer, sheet_name='Pipe_month', index=False)

pivot_month_pipe_conf = df10.pivot_table(index=['regionName', 'pipelineName', 'direction', 'startPumpStation',
                                         'finishPumpStation'], columns=df10['reportDate'].dt.date, values='confidenceId').reset_index()
pivot_month_pipe_conf.to_excel(writer, sheet_name='Pipe_Conf', index=False)

pivot_month_pipe_cap = df10.pivot_table(index=['regionName', 'pipelineName', 'direction', 'startPumpStation',
                                        'finishPumpStation'], columns=df10['reportDate'].dt.date, values='pipelineCapacity').reset_index()
pivot_month_pipe_cap.to_excel(writer, sheet_name='Pipe_Cap', index=False)

pivot_daily_pipe_flow = df5.pivot_table(index=['regionName', 'pipelineName', 'direction', 'startPumpStation',
                                        'finishPumpStation'], columns=df5['reportDate'].dt.date, values='flowBpd').reset_index()
pivot_daily_pipe_flow.to_excel(writer, sheet_name='Pipe_Day', index=False)

pivot_stor_reg = df_stor_reg.pivot_table(
    index=['region', 'product'], columns=df_stor_reg['reportDate'].dt.date, values='storageAmount').reset_index()
pivot_stor_reg.to_excel(writer, sheet_name='stor_reg', index=False)

pivot_stor_reg_cap = df_stor_reg.pivot_table(
    index=['region', 'product'], columns=df_stor_reg['reportDate'].dt.date, values='operationalCapacity').reset_index()
pivot_stor_reg_cap.to_excel(writer, sheet_name='stor_reg_cap', index=False)

pivot_stor_own = df_stor_own.pivot_table(index=['region', 'storageField', 'owner', 'product'],
                                         columns=df_stor_own['reportDate'].dt.date, values='storageAmount').reset_index()
pivot_stor_own.to_excel(writer, sheet_name='stor_own', index=False)

pivot_stor_own_cap = df_stor_own.pivot_table(index=['region', 'storageField', 'owner', 'product'],
                                             columns=df_stor_own['reportDate'].dt.date, values='operationalCapacity').reset_index()
pivot_stor_own_cap.to_excel(writer, sheet_name='stor_own_cap', index=False)

df_meta.to_excel(writer, sheet_name='Meta', index=False)

writer.save()


### xlsx file for model usage

In [16]:
url = 'https://raw.githubusercontent.com/patdat/GSPE/main/Genscape%20Model%20List.csv'
df_model_pipe = pd.read_csv(url)


In [17]:
def model_pivot_pipe(x, y):
    df = pd.merge(x, y, 'left', on=['pipelineName', 'direction'])
#    df.reset_index( inplace=True)
    df = df.drop(['regionName', 'startPumpStation',
                 'finishPumpStation'], axis=1)
    df = df.drop_duplicates(subset=['pipelineName', 'direction'], keep='last')

    return df


df_model_pipe_flow = model_pivot_pipe(df_model_pipe, pivot_month_pipe_flow)
df_model_pipe_flow.iloc[:, 2:] = df_model_pipe_flow.iloc[:, 2:].div(
    1000, axis=0)

df_model_pipe_conf = model_pivot_pipe(df_model_pipe, pivot_month_pipe_conf)

df_model_pipe_cap = model_pivot_pipe(df_model_pipe, pivot_month_pipe_cap)
df_model_pipe_cap.iloc[:, 2:] = df_model_pipe_cap.iloc[:, 2:].div(1000, axis=0)


GENSCAPE_XLSX = 'genscape_data_model.xlsx'
writer = pd.ExcelWriter(GENSCAPE_XLSX, engine='xlsxwriter')

df_model_pipe_flow.to_excel(writer, sheet_name='Pipe_Flow', index=False)
df_model_pipe_conf.to_excel(writer, sheet_name='Pipe_Conf', index=False)
df_model_pipe_cap.to_excel(writer, sheet_name='Pipe_Cap', index=False)

writer.save()
